# Dependencies

I believe `requests` and `pandas` are likely pre-installed, though `plotly` and `statsmodels` may need to be added...

    # pip or pip3 depending on setup
    pip3 install plotly statsmodels

# Acquire Data

Pull data from the URL referenced on the [coronavirus.maryland.gov](https://coronavirus.maryland.gov/) site.  Looking for a 200 HTTP Status code for successful pull.

In [ ]:
import requests
import pandas
import json
import plotly.express as px
import plotly.graph_objects as go

pandas.options.display.max_rows = None

params = {
    "f": "json",
    "where": "1=1",
    "returnGeometry": "false",
    "spatialRel": "esriSpatialRelIntersects",
    "outFields": "*",
    "orderByFields": "ReportDate asc",
    "resultOffset": "0",
    "resultRecordCount": "32000",
    "resultType": "standard",
    "cacheHint": "true"
}

URL = 'https://services.arcgis.com/njFNhDsUCentVYJW/arcgis/rest/services/MASTER_CaseTracker_3/FeatureServer/0/query'
            
response = requests.request("GET", URL, params=params)

# 200 status means we're good to go
print("HTTP Status: {}".format(response.status_code))

# Keys

Notable keys/values...
  * `fields` - semi-definition of each value (maybe makes sense in ESRI land?)
  * `features` - this is where the real daily data lies

In [ ]:
payload = response.json()

# pandas only used for easy display
display(pandas.DataFrame(payload.keys()))

# Actual Data

Once flattened, you can see what data is avaialble for each day...

In [ ]:
# flatten a bit, as each item is rooted under an uncessary attributes key 
daily_numbers = [element['attributes'] for element in payload['features']]

# pandas only used for easy display
display(pandas.DataFrame(daily_numbers[0].keys()))

In [ ]:
# slurp into dataframe
df = pandas.read_json(json.dumps(daily_numbers), orient='records')

In [ ]:
# df_melt = filtered.melt(id_vars='ReportDate', value_vars=['bedsAcute', 'bedsICU', 'bedsTotal'])
# fig = px.line(df_melt, x='ReportDate' , y='value', color='variable')
# fig.show()

In [ ]:
filtered = df.filter(items=['ReportDate', 'bedsAcute', 'bedsICU', 'bedsTotal'])
filtered['ReportDate'] = pandas.to_datetime(df['ReportDate'], unit='ms')

fig = go.Figure()
for x in ['bedsAcute', 'bedsICU', 'bedsTotal']:
    fig.add_trace(go.Scatter(x=filtered['ReportDate'], y=df[x], mode='lines+markers', name=x))
fig.show()

In [ ]:
filtered = df.filter(items=['ReportDate', 'bedsAcute', 'bedsICU', 'bedsTotal'])
# for whatever reason lowess doesn't like datetimes?
# filtered['ReportDate'] = pandas.to_datetime(df['ReportDate'], unit='ms')

df_melt = filtered.melt(id_vars='ReportDate', value_vars=['bedsAcute', 'bedsICU', 'bedsTotal'])
fig = px.scatter(df_melt, x='ReportDate' , y='value', color='variable', trendline="lowess")
fig.show()

In [ ]:
filtered = df.filter(items=['ReportDate', 'total_hospitalized', 'hospitalizedDelta'])
filtered['ReportDate'] = pandas.to_datetime(df['ReportDate'], unit='ms')
display(filtered)

In [ ]:
fig = go.Figure()
for x in ['hospitalizedDelta']:
    fig.add_trace(go.Scatter(x=filtered['ReportDate'], y=filtered[x], mode='lines+markers', name=x))
fig.show()